### 전일대비 거래량 변화율

In [1]:
import pandas as pd
import os

# CSV 파일들이 있는 폴더 경로를 지정하세요
folder_path = 'C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/주가'  # 예: 'C:/Users/username/Documents/stock_data'

# 폴더 내 모든 .csv 파일 순회
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        
        try:
            # CSV 파일 읽기
            df = pd.read_csv(file_path)

            # '거래량' 열이 있는 경우에만 처리
            if '거래량' in df.columns:
                # 거래량 변화율 (전일 대비 변화 비율) 추가 — 백분율 변환 없음
                df['거래량 변화율'] = df['거래량'].pct_change()

                # 덮어쓰기 저장
                df.to_csv(file_path, index=False)
                print(f"{filename} 처리 완료")
            else:
                print(f"{filename}에는 '거래량' 열이 없음 - 건너뜀")

        except Exception as e:
            print(f"{filename} 처리 중 오류 발생: {e}")


HYBE.csv 처리 완료
HYBE_시가총액.csv 처리 완료
JYP.csv 처리 완료
JYP_시가총액.csv 처리 완료
KOSDAQ Entertainment & Culture.csv 처리 완료
RBW.csv 처리 완료
RBW_시가총액.csv 처리 완료
SM C&C.csv 처리 완료
SM C&C_시가총액.csv 처리 완료
SM.csv 처리 완료
SM_시가총액.csv 처리 완료
YG.csv 처리 완료
YG_시가총액.csv 처리 완료
아티스트컴퍼니.csv 처리 완료
아티스트컴퍼니_시가총액.csv 처리 완료
코스닥.csv 처리 완료
큐브.csv 처리 완료
큐브_시가총액.csv 처리 완료
키이스트.csv 처리 완료
키이스트_시가총액.csv 처리 완료


In [7]:
import pandas as pd
import numpy as np
import os

# 1. 사건 데이터 불러오기
scandal = pd.read_csv("C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/스캔들/ALL논란_Final_시가총액.csv", encoding='utf-8', parse_dates=['사건 날짜'])

# 2. 날짜 매핑 함수: 사건 이후 가장 가까운 날짜
def map_closest_date(scandal_date, stock_dates):
    return stock_dates[stock_dates >= scandal_date].min()

# 3. 결과 저장 리스트
mapped_dates = []
volume_changes = []

# 4. 절대 경로 설정
stock_data_dir = r"C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/주가"

# 5. 각 사건 행에 대해 처리
for idx, row in scandal.iterrows():
    firm = row['소속사']
    scandal_date = row['사건 날짜']
    
    stock_file = os.path.join(stock_data_dir, f"{firm}.csv")
    
    mapped_date = pd.NaT
    volume_change = np.nan

    if not os.path.exists(stock_file):
        print(f"❗ 주가 파일 없음: {stock_file}")
        mapped_dates.append(mapped_date)
        volume_changes.append(volume_change)
        continue

    try:
        # 주가 데이터 불러오기 및 정렬
        stock = pd.read_csv(stock_file, parse_dates=['날짜'])
        stock = stock.sort_values('날짜').reset_index(drop=True)

        # 거래량 변화율 계산 (비율 형태)
        stock['거래량_변화율'] = stock['거래량'].pct_change()

        # 사건 이후 가장 가까운 날짜 매핑
        mapped_date = map_closest_date(scandal_date, stock['날짜'])
        mapped_dates.append(mapped_date)

        if pd.isna(mapped_date):
            volume_changes.append(np.nan)
            continue

        # 매핑된 날짜의 거래량 변화율 추출
        match_row = stock[stock['날짜'] == mapped_date]
        if not match_row.empty:
            volume_change = match_row.iloc[0]['거래량_변화율']
        
        volume_changes.append(volume_change)

    except Exception as e:
        print(f"❌ {firm} 처리 중 오류: {e}")
        mapped_dates.append(mapped_date)
        volume_changes.append(np.nan)

# 6. 결과 열 추가
scandal['전일 대비 거래량 변화율'] = volume_changes

# 7. 저장
output_path = "ALL논란_Final_시가총액_거래량변화율.csv"
scandal.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"✅ 저장 완료: {output_path}")

✅ 저장 완료: ALL논란_Final_시가총액_거래량변화율.csv


In [8]:
sum(scandal['전일 대비 거래량 변화율'].isna())

0

심호오빠 데이터에 거래량 열 추가

In [ ]:
import pandas as pd
import numpy as np
import os

# 1. 사건 데이터 불러오기
scandal = pd.read_csv("C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/이벤트 스터디 결과/Final_라벨결측제거_원핫인코딩_직업.csv", encoding='utf-8', parse_dates=['사건 날짜'])

# 2. 날짜 매핑 함수: 사건 이후 가장 가까운 날짜
def map_closest_date(scandal_date, stock_dates):
    return stock_dates[stock_dates >= scandal_date].min()

# 3. 결과 저장 리스트
mapped_dates = []
volume_changes = []

# 4. 절대 경로 설정
stock_data_dir = r"C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/주가"

# 5. 각 사건 행에 대해 처리
for idx, row in scandal.iterrows():
    firm = row['소속사']
    scandal_date = row['사건 날짜']
    
    stock_file = os.path.join(stock_data_dir, f"{firm}.csv")
    
    mapped_date = pd.NaT
    volume_change = np.nan

    if not os.path.exists(stock_file):
        print(f"❗ 주가 파일 없음: {stock_file}")
        mapped_dates.append(mapped_date)
        volume_changes.append(volume_change)
        continue

    try:
        # 주가 데이터 불러오기 및 정렬
        stock = pd.read_csv(stock_file, parse_dates=['날짜'])
        stock = stock.sort_values('날짜').reset_index(drop=True)

        # 거래량 변화율 계산 (비율 형태)
        stock['거래량_변화율'] = stock['거래량'].pct_change()

        # 사건 이후 가장 가까운 날짜 매핑
        mapped_date = map_closest_date(scandal_date, stock['날짜'])
        mapped_dates.append(mapped_date)

        if pd.isna(mapped_date):
            volume_changes.append(np.nan)
            continue

        # 매핑된 날짜의 거래량 변화율 추출
        match_row = stock[stock['날짜'] == mapped_date]
        if not match_row.empty:
            volume_change = match_row.iloc[0]['거래량_변화율']
        
        volume_changes.append(volume_change)

    except Exception as e:
        print(f"❌ {firm} 처리 중 오류: {e}")
        mapped_dates.append(mapped_date)
        volume_changes.append(np.nan)

# 6. 결과 열 추가
scandal['전일 대비 거래량 변화율'] = volume_changes

# 7. 저장
output_path = "Final_라벨결측제거_원핫인코딩_직업.csv"
scandal.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"✅ 저장 완료: {output_path}")

✅ 저장 완료: 사건별_정당성_검정결과_tmp_거래량변화율.csv


In [2]:
sum(scandal['전일 대비 거래량 변화율'].isna())

0

In [1]:
import pandas as pd
from tqdm import tqdm

# 데이터 불러오기
df = pd.read_csv("Final_라벨결측제거_원핫인코딩_직업.csv", parse_dates=['사건 날짜(매핑된 주가 날짜)'])

# 시가총액을 저장할 리스트
market_caps = []

# 각 행마다 소속사와 날짜 기준으로 시가총액 매핑
for i in tqdm(range(len(df))):
    firm = df.loc[i, '소속사']
    date = df.loc[i, '사건 날짜(매핑된 주가 날짜)']
    
    try:
        # 시가총액 파일 경로 (파일명은 소속사명 기반)
        file_path = f"C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/주가/{firm}.csv"  # 실제 경로로 수정 필요
        cap_df = pd.read_csv(file_path, parse_dates=['날짜'])

        # 해당 날짜 행 찾기
        row = cap_df.loc[cap_df['날짜'] == date]

        if not row.empty:
            market_caps.append(row.iloc[0]['거래량 변화율'])
        else:
            market_caps.append(None)

    except FileNotFoundError:
        print(f"{file_path} 파일이 존재하지 않습니다.")
        market_caps.append(None)

# 결과 컬럼 추가
df['거래량 변화율'] = market_caps

100%|██████████| 253/253 [00:01<00:00, 146.56it/s]


In [3]:
sum(df['거래량 변화율'].isna())

0

In [4]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', '소속사', '연예인 이름', '사건 날짜', 'AR1', 'AR2',
       '그룹', '사건 내용', '라벨 1', '라벨 2', '라벨 3', '사건 날짜(매핑된 주가 날짜)', 'CAR(1,2)',
       '시가총액', '년월', '관심도', 'log_시가총액', '시가총액_랭크', 'stock_trend_n10', '사생활',
       '사회적 감수성', '종교/이념', '범죄 혐의', '발언 문제', '혐의정보 유포', '세금 문제', '무혐의', '팬 대응',
       '무의식적 태도', '병역 문제', '기타', '성 관련', '직업', '직업_배우', '직업_아이돌', '거래량 변화율'],
      dtype='object')

In [5]:
df.to_csv('Final_라벨결측제거_원핫인코딩_직업.csv', encoding='utf-8-sig')